## Findings

1. The top five schools are charter schools. The bottom five schools are district schools. 
2. The largest schools have the lowest average overall passing rates. The small and medium schools have the higest average overall passing rates.
3. The two school groups spending least actually have a significantly higher overall passing rate than the two school groups spending the most. 

In [1]:
# Dependencies 
import pandas as pd

# Load files and read into data frames
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [2]:
# Calculate the total number of schools and total budget
unique_schools = school_data_complete['school_name'].nunique()
total_budget = school_data['budget'].sum()

# Calculate the total number of students, total budget, average math score, and average reading score
school_describe = school_data_complete.describe(percentiles=[])

In [3]:
school_describe

,Student ID,reading_score,math_score,School ID,size,budget
count,39170.000000,39170.00000,39170.000000,39170.000000,39170.000000,3.917000e+04
mean,19584.500000,81.87784,78.985371,6.978172,3332.957110,2.117241e+06
std,11307.549359,10.23958,12.309968,4.444329,1323.914069,8.749987e+05
min,0.000000,63.00000,55.000000,0.000000,427.000000,2.480870e+05
50%,19584.500000,82.00000,79.000000,7.000000,2949.000000,1.910635e+06
max,39169.000000,99.00000,99.000000,14.000000,4976.000000,3.124928e+06


In [4]:
# Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
passing_rate = (school_data_complete["math_score"].mean() + school_data_complete["reading_score"].mean()) / 2

# Calculate the percentage of students with a passing math score and passing reading score (70 or greater)
# Create and append our bins
grade_bins = [0, 70, 100]
grade_labels = ["Failing", "Passing"]
school_data_complete["math_status"] = pd.cut(school_data_complete["math_score"], grade_bins, labels=grade_labels)
school_data_complete["reading_status"] = pd.cut(school_data_complete["reading_score"], grade_bins, labels=grade_labels)

#Print percentage of passing and failing students in each subject
math_passfail = school_data_complete["math_status"].value_counts(normalize=True)
reading_passfail = school_data_complete["reading_status"].value_counts(normalize=True)

In [5]:
math_passfail

Passing    0.723921
Failing    0.276079
Name: math_status, dtype: float64

In [6]:
reading_passfail

Passing    0.829717
Failing    0.170283
Name: reading_status, dtype: float64

## District summary

In [7]:
# Create a dataframe to hold the above results
raw_dict = {
    "Unique schools": [unique_schools],            
    "Total students": [school_describe.iloc[0 , 0]],
    "Total budget": [total_budget],
    "Avg. math score": [school_describe.iloc[1 , 2]],
    "Avg. reading score": [school_describe.iloc[1 , 1]],
    "Math pass": [math_passfail[1]],
    "Math fail": [math_passfail[0]],
    "Reading pass": [reading_passfail[1]],
    "Reading fail": [reading_passfail[0]]        
           }
district_summary = pd.DataFrame(raw_dict)

In [8]:
# Cleaner formatting for displayed data
district_summary.style.hide_index().format({
    'Total students': '{:,.0f}',
    'Total budget': '${:,.0f}',
    'Avg. math score': '{:,.2f}',
    'Avg. reading score': '{:,.2f}',
    'Math pass': '{:,.2%}',
    'Math fail': '{:,.2%}',
    'Reading pass': '{:,.2%}',
    'Reading fail': '{:,.2%}'
})

Unique schools,Total students,Total budget,Avg. math score,Avg. reading score,Math pass,Math fail,Reading pass,Reading fail
15,"39,170","$24,649,428",78.99,81.88,72.39%,27.61%,82.97%,17.03%


## School summary

In [9]:
# Group by the school names
school_grouped = school_data_complete.groupby('school_name')

# Overview table summarizing key metrics about each school, including school name, average math score, average reading score, total students
average_scores = school_grouped["reading_score", "math_score"].mean()
total_students = school_grouped['Student ID'].count()
school_summary = pd.merge(average_scores, total_students, on="school_name")
school_summary['School type'] = school_grouped["type"].max()
school_summary['Total budget'] = school_grouped["budget"].max()
school_summary['Per student budget'] = school_summary['Total budget']/school_summary['Student ID']
school_summary['passingmath'] = school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name')['Student ID'].count()
school_summary['passingreading'] = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('school_name')['Student ID'].count()
school_summary['Percent passing math'] = school_summary["passingmath"]/school_summary["Student ID"]
school_summary['Percent passing reading'] = school_summary["passingreading"]/school_summary["Student ID"]
school_summary['Overall passing rate'] = (school_summary["Percent passing math"]+school_summary["Percent passing reading"]) / 2

In [10]:
school_summary.rename(columns = {
    'reading_score':'Avg. reading score', 
    'math_score':'Avg. math score',
    'Student ID':'Total students'                                 
}, inplace = True) 
school_summary.index.name = "School"

In [11]:
# Delete some stuff
school_summary.pop('passingmath')
school_summary.pop('passingreading')

School
Bailey High School       4077
Cabrera High School      1803
Figueroa High School     2381
Ford High School         2172
Griffin High School      1426
Hernandez High School    3748
Holden High School        411
Huang High School        2372
Johnson High School      3867
Pena High School          923
Rodriguez High School    3208
Shelton High School      1688
Thomas High School       1591
Wilson High School       2204
Wright High School       1739
Name: passingreading, dtype: int64

In [12]:
# Rearrange
school_summary = school_summary[[
    'School type',
    'Total students',
    'Total budget',
    'Per student budget',
    'Avg. reading score',
    'Avg. math score',
    'Percent passing math',
    'Percent passing reading',
    'Overall passing rate'    
]]

# Cleaner formatting for displayed data
school_summary.style.format({
    'Total students': '{:,}', 
    'Total budget': '${:,}',
    'Per student budget': '${:.2f}',
    'Avg. math score': '{:.2f}', 
    'Avg. reading score': '{:.2f}', 
    'Percent passing math': '{:.2%}', 
    'Percent passing reading': '{:.2%}', 
    'Overall passing rate': '{:.2%}'
})

,School type,Total students,Total budget,Per student budget,Avg. reading score,Avg. math score,Percent passing math,Percent passing reading,Overall passing rate
School,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928",$628.00,81.03,77.05,66.68%,81.93%,74.31%
Cabrera High School,Charter,"1,858","$1,081,356",$582.00,83.98,83.06,94.13%,97.04%,95.59%
Figueroa High School,District,"2,949","$1,884,411",$639.00,81.16,76.71,65.99%,80.74%,73.36%
Ford High School,District,"2,739","$1,763,916",$644.00,80.75,77.10,68.31%,79.30%,73.80%
Griffin High School,Charter,"1,468","$917,500",$625.00,83.82,83.35,93.39%,97.14%,95.27%
Hernandez High School,District,"4,635","$3,022,020",$652.00,80.93,77.29,66.75%,80.86%,73.81%
Holden High School,Charter,427,"$248,087",$581.00,83.81,83.80,92.51%,96.25%,94.38%
Huang High School,District,"2,917","$1,910,635",$655.00,81.18,76.63,65.68%,81.32%,73.50%
Johnson High School,District,"4,761","$3,094,650",$650.00,80.97,77.07,66.06%,81.22%,73.64%


## Top performing schools (by passing rate)

In [13]:
# Sort and display the top five schools in overall passing rate
# Cleaner formatting for displayed data
school_summary.sort_values(by=['Overall passing rate'], ascending = False).head().style.format({
    'Total students': '{:,}', 
    'Total budget': '${:,}',
    'Per student budget': '${:.2f}',
    'Avg. math score': '{:.2f}', 
    'Avg. reading score': '{:.2f}', 
    'Percent passing math': '{:.2%}', 
    'Percent passing reading': '{:.2%}', 
    'Overall passing rate': '{:.2%}'
})

,School type,Total students,Total budget,Per student budget,Avg. reading score,Avg. math score,Percent passing math,Percent passing reading,Overall passing rate
School,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356",$582.00,83.98,83.06,94.13%,97.04%,95.59%
Thomas High School,Charter,"1,635","$1,043,130",$638.00,83.85,83.42,93.27%,97.31%,95.29%
Pena High School,Charter,962,"$585,858",$609.00,84.04,83.84,94.59%,95.95%,95.27%
Griffin High School,Charter,"1,468","$917,500",$625.00,83.82,83.35,93.39%,97.14%,95.27%
Wilson High School,Charter,"2,283","$1,319,574",$578.00,83.99,83.27,93.87%,96.54%,95.20%


## Bottom performing schools (by passing rate)

In [14]:
# Sort and display the top five schools in overall passing rate
# Cleaner formatting for displayed data
school_summary.sort_values(by=['Overall passing rate'], ascending = True).head().style.format({
    'Total students': '{:,}', 
    'Total budget': '${:,}',
    'Per student budget': '${:.2f}',
    'Avg. math score': '{:.2f}', 
    'Avg. reading score': '{:.2f}', 
    'Percent passing math': '{:.2%}', 
    'Percent passing reading': '{:.2%}', 
    'Overall passing rate': '{:.2%}'
})

,School type,Total students,Total budget,Per student budget,Avg. reading score,Avg. math score,Percent passing math,Percent passing reading,Overall passing rate
School,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363",$637.00,80.74,76.84,66.37%,80.22%,73.29%
Figueroa High School,District,"2,949","$1,884,411",$639.00,81.16,76.71,65.99%,80.74%,73.36%
Huang High School,District,"2,917","$1,910,635",$655.00,81.18,76.63,65.68%,81.32%,73.50%
Johnson High School,District,"4,761","$3,094,650",$650.00,80.97,77.07,66.06%,81.22%,73.64%
Ford High School,District,"2,739","$1,763,916",$644.00,80.75,77.10,68.31%,79.30%,73.80%


## Math scores by grade

In [15]:
#Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
#Create a pandas series for each grade
#Group each series by school, combine series into data frame
school_math_scores = pd.DataFrame({
    "9th grade": school_data_complete.loc[school_data_complete['grade'] == '9th'].groupby('school_name')['math_score'].mean(),
    "10th grade": school_data_complete.loc[school_data_complete['grade'] == '10th'].groupby('school_name')['math_score'].mean(),
    "11th grade": school_data_complete.loc[school_data_complete['grade'] == '11th'].groupby('school_name')['math_score'].mean(),
    "12th grade": school_data_complete.loc[school_data_complete['grade'] == '12th'].groupby('school_name')['math_score'].mean()
})

# Cleaner title and formatting for displayed data
school_math_scores.index.name = "School"
school_math_scores.style.format({
    '9th grade': '{:.2f}',
    '10th grade': '{:.2f}',
    '11th grade': '{:.2f}',
    '12th grade': '{:.2f}'    
})

,9th grade,10th grade,11th grade,12th grade
School,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading score by grade 

In [16]:
# Perform the same operations as above for reading scores
school_reading_scores = pd.DataFrame({
    "9th grade": school_data_complete.loc[school_data_complete['grade'] == '9th'].groupby('school_name')['reading_score'].mean(),
    "10th grade": school_data_complete.loc[school_data_complete['grade'] == '10th'].groupby('school_name')['reading_score'].mean(),
    "11th grade": school_data_complete.loc[school_data_complete['grade'] == '11th'].groupby('school_name')['reading_score'].mean(),
    "12th grade": school_data_complete.loc[school_data_complete['grade'] == '12th'].groupby('school_name')['reading_score'].mean()
})

# Cleaner title and formatting for displayed data
school_reading_scores.index.name = "School"
school_reading_scores.style.format({
    '9th grade': '{:.2f}',
    '10th grade': '{:.2f}',
    '11th grade': '{:.2f}',
    '12th grade': '{:.2f}'    
})

,9th grade,10th grade,11th grade,12th grade
School,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by school spending

In [17]:
# Four bins to group school spending
# Sample bins
spending_bins = [0, 585, 615, 645, 675]
spending_bin_names = ["<$585", "$585-615", "$615-645", "$645-675"]
school_summary['Spending bin'] = pd.cut(school_summary['Per student budget'], spending_bins, labels = spending_bin_names)

# Bin the schools
spending_bin_school_summary = school_summary.groupby('Spending bin')

# New data frame, including average math score, average reading score, % passing math, % passing reading, overall passing rate (average of previous two
spending_grouped_school_summary = pd.DataFrame({
    "Avg. math score": spending_bin_school_summary['Avg. math score'].mean(),
    "Avg. reading score": spending_bin_school_summary['Avg. reading score'].mean(),
    "Percent passing math": spending_bin_school_summary['Percent passing math'].mean(),
    "Percent passing reading": spending_bin_school_summary['Percent passing reading'].mean(),
    "Overall passing rate": spending_bin_school_summary['Overall passing rate'].mean()
})

# Cleaner title and formatting for displayed data
spending_grouped_school_summary.index.name = "Group (spending)"
spending_grouped_school_summary.style.format({
    'Avg. math score': '{:.2f}', 
    'Avg. reading score': '{:.2f}', 
    'Percent passing math': '{:.2%}', 
    'Percent passing reading': '{:.2%}', 
    'Overall passing rate': '{:.2%}'
})

,Avg. math score,Avg. reading score,Percent passing math,Percent passing reading,Overall passing rate
Group (spending),,,,,
<$585,83.46,83.93,93.46%,96.61%,95.04%
$585-615,83.60,83.89,94.23%,95.90%,95.07%
$615-645,79.08,81.89,75.67%,86.11%,80.89%
$645-675,77.00,81.03,66.16%,81.13%,73.65%


## Scores by school size

In [18]:
# Same operations as above based on school size
size_bins = [0, 1000, 2000, 5000]
size_bin_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_summary['Size bin'] = pd.cut(school_summary['Total students'], size_bins, labels = size_bin_names)

# Bin the schools
size_bin_school_summary = school_summary.groupby('Size bin')

# Make a new data frame, include the previous
size_grouped_school_summary = pd.DataFrame({
    "Avg. math score": size_bin_school_summary['Avg. math score'].mean(),
    "Avg. reading score": size_bin_school_summary['Avg. reading score'].mean(),
    "Percent passing math": size_bin_school_summary['Percent passing math'].mean(),
    "Percent passing reading": size_bin_school_summary['Percent passing reading'].mean(),
    "Overall passing rate": size_bin_school_summary['Overall passing rate'].mean()
})

# Cleaner title and formatting for displayed data
size_grouped_school_summary.index.name = "Group (size)"
size_grouped_school_summary.style.format({
    'Avg. math score': '{:.2f}', 
    'Avg. reading score': '{:.2f}', 
    'Percent passing math': '{:.2%}', 
    'Percent passing reading': '{:.2%}', 
    'Overall passing rate': '{:.2%}'
})

,Avg. math score,Avg. reading score,Percent passing math,Percent passing reading,Overall passing rate
Group (size),,,,,
Small (<1000),83.82,83.93,93.55%,96.10%,94.82%
Medium (1000-2000),83.37,83.86,93.60%,96.79%,95.20%
Large (2000-5000),77.75,81.34,69.96%,82.77%,76.36%


## Scores by school type

In [19]:
# Same operations as above based on school type
type_bin_school_summary = school_summary.groupby('School type')

# Make a new data frame, include the previous
type_grouped_school_summary = pd.DataFrame({
    "Avg. math score": type_bin_school_summary['Avg. math score'].mean(),
    "Avg. reading score": type_bin_school_summary['Avg. reading score'].mean(),
    "Percent passing math": type_bin_school_summary['Percent passing math'].mean(),
    "Percent passing reading": type_bin_school_summary['Percent passing reading'].mean(),
    "Overall passing rate": type_bin_school_summary['Overall passing rate'].mean()
})

# Cleaner title and formatting for displayed data
type_grouped_school_summary.index.name = "Group (type)"
type_grouped_school_summary.style.format({
    'Avg. math score': '{:.2f}', 
    'Avg. reading score': '{:.2f}', 
    'Percent passing math': '{:.2%}', 
    'Percent passing reading': '{:.2%}', 
    'Overall passing rate': '{:.2%}'
})

,Avg. math score,Avg. reading score,Percent passing math,Percent passing reading,Overall passing rate
Group (type),,,,,
Charter,83.47,83.90,93.62%,96.59%,95.10%
District,76.96,80.97,66.55%,80.80%,73.67%
